# La recherche textuelle

## Algorithme de Boyer-Moore de recherche textuelle

In [2]:
# texte
motif="AT-THAT"
texte="WHICH-FINALLY-HALTS.--AT-THAT-POINT"
alphabet="WHIC-FNALYTS.O"

### Retour sur la recherche simple

Le script ci-dessous permet d'effectuer une recherche simple. i pointe un caractère du texte. j pointe un caractère du motif. Après chaque décalage du motif d'une case vers la droite, i et j sont initialisés à la fin du motif pour j et sur la lettre du texte coincidant avec la dernière lettre du motif pour i. La comparaison se fait à partir de la fin du motif et en remontant la chaîne.

In [3]:
def recherche(motif,texte):
    
    n=len(texte)
    m=len(motif)
    p=[]
    
    i=m-1
    while i<n:
        j=m-1
        while j>-1 and texte[i]==motif[j]:
            j=j-1 
            i=i-1
            
        if j==-1:
            p.append(i+1)
            i=i+(m-j)
        else:
            i=i+(m-j) # m-j permet d'avancer le motif de 1 case vers la droite
    return p

### Algorithme de boyer moore limité à la régle du bon suffixe

L'algorithme de boyer moore parcours les lettres de la même manière que notre algorithme de recherche précédent. La seule différence est que quand deux lettres ne correspondent pas; le décalage est optimisé ie il est plus grand que celui de la recherche simple.

Le décalage est pré-calculé (valeurs dans des tableaux d1 et d2) à partir du MOTIF selon deux règles : la règle du mauvais caractère et la régle du bon suffixe. Dans ce notebook, on se limite à la règle du bon suffixe.


### Présentation de la règle du bon suffixe

Reprenons l'exemple utilisant la règle du mauvais caractère :

```
WHICH-FINALLY-HALTS.--AT-THAT-POINT
AT-THAT

WHICH-FINALLY-HALTS.--AT-THAT-POINT
       AT-THAT

WHICH-FINALLY-HALTS.--AT-THAT-POINT
           AT-THAT

WHICH-FINALLY-HALTS.--AT-THAT-POINT
                 AT-THAT
                 
A ce point, on pourrait aligner le - du motif avec le - au dessus du H. Mais, il est possible de faire mieux :
aligner le AT du début du motif avec le AT du texte. Au lieu d'aller chercher des alignements avec des lettres,
on cherche un alignement avec un groupe de lettre : le suffixe du motif qui est aligné avec des lettres du texte.
Il s'agit de la règle du bon suffixe.

On arrive alors directement sur la situation suivante

WHICH-FINALLY-HALTS.--AT-THAT-POINT
                      AT-THAT
```

In [4]:
def coincidence(motif,j,k,longueur):
    i=longueur-1
    while i>j and (motif[i]==motif[k+i-j] or motif[k+i-j]=="$"):
        i=i-1
    if i==j:
        return True
    else:
        return False

In [5]:
def pretraitement_BS(motif):
    m=len(motif)
    d=[0 for i in range(m)]
    d[m-1]=1
    motif=motif+"$"*(m+1)
    for j in range(0,m-1):
        for k in range(j-m,j):
            if coincidence(motif,j,k,m) and motif[k]!=motif[j]:
                d[j]=m-k-1
    return d

### Comment réaliser la table de saut selon le critère du bon suffixe ?

Cette table s'appelle d2. On suppose qu'un mauvais caractère arrive à la position j. Avant la position j, un certain nombre de caractère correspondent au texte; ces caractères forme le suffixe.

d2[j] est la distance dont nous devons décaler le motif vers la droite pour aligner les caractères formant le suffixe du motif (m-j caractères pour ce motif) avec la plus plausible et à droite réoccurence de ce suffixe dans le motif plus la distance additionnelle pour recommencer le processus à la droite du motif. d2[j] indique alors de combien de case faut-il décaler i.

```
Exemple de AT-THAT

         i      
...XXXXXXX...      
   AT-THAT
         j
   
La table a pour entrée j : la position pour laquelle les lettres ne correspondent plus. Les lettres qui correspondent sont appelées suffixe.

Exemple pour j=6

         i      
...XXXXXXX...      
   AT-THAT
         j=6

T ne se retrouve pas dans le texte en position i donc le motif doit être décalé d'une seule case (on ne peut faire autrement)

          i      
...XXXXXXX...      
    AT-THAT

d2[6]=1 D'une manière générale la dernière case de d2 vaut 1 (on se ramène ainsi à la recherche simple)

Exemple pour j=5

        i      
...XXXXXXT...      
   AT-THAT
        j=5

Le T correspond mais le A non donc on cherche le groupe T le plus à droite (en évitant qu'il commence par un A)
D'une manière générale, le caractère juste avant le suffixe ne doit pas se trouver avant le groupe recherché. Le T précédé d'un - va donc bien

        i   i   
...XXXXXXT...      
      AT-THAT

i doit être décalé de 4 cases
d2[5]=4

Exemple pour j=4

       i      
...XXXXXAT...      
   AT-THAT
       j=4
       
Le suffixe AT correspond mais le H précédent non. On cherche le groupe AT le plus à droite (non précédé d'un H).
Il y a bien AT en début de motif mais sans caractère avant. Et bien il faut considérer qu'avant le début du motif, en position négative, il y a des caractères qu'on notera § et qui jouent le rôle de joker : § n'est pas un H

       i      i      
...XXXXXAT...      
       §AT-THAT
       j=4

i doit être décalé de 7 cases
d2[4]=7

Exemple pour j=3

      i      
...XXXXHAT...      
   AT-THAT
      j=3

Le suffixe HAT correspond mais le T précédent non. On cherche le groupe HAT le plus à droite non précédé d'un T. On va encore utiliser les joker § : § peut remplacer n'importe quelle lettre et § n'est pas un T

      i       i
...XXXXHAT...      
      §§AT-THAT
      j=3
      
i doit être décalé de 8 cases
d2[3]=8

Exemple pour j=0

   i      
...XT-THAT...      
   AT-THAT
   j=0
   
Le suffixe T-THAT correspond mais le A précédent non. On cherche le groupe T-THAT le plus à droite non précédé d'un A. On va encore utiliser les joker § : § peut remplacer n'importe quelle lettre et § n'est pas un A

   i          i      
...XT-THAT...      
    §§§§AT-THAT
   j=0
   
i doit être décalé de 11 cases
d2[0]=11

En résumé,

j     Suffixe            Décalage de i
0     T-THAT             11
1     -THAT              10
2     THAT               9
3     HAT                8
4     AT                 7
5     T                  4
6    pas de suffixe      1
```

In [6]:
def recherche(motif,texte,alphabet):
    
    # d1=pretraitement_MC(motif,alphabet)
    d2=pretraitement_BS(motif)
    
    n=len(texte)
    m=len(motif)
    
    i=m-1
    while i<n:
        print(texte)
        print(" "*(i-m+1)+motif)
        print()
        j=m-1
        while j>-1 and texte[i]==motif[j]:
            j=j-1
            i=i-1
            
        if j==-1:
            return i+1,True
        else:
            i=i+d2[j]
            # i=i+max(d1[texte[i]],d2[j]) pour utiliser les deux règles
    return i+1,False

In [8]:
# print(motif)
# print(texte)
recherche(motif,texte,alphabet) # dans la majorité des cas le décalage n'est que de 1.
                                # On voit l'interêt d'ajouter la règle du mauvais caractère.


WHICH-FINALLY-HALTS.--AT-THAT-POINT
AT-THAT

WHICH-FINALLY-HALTS.--AT-THAT-POINT
 AT-THAT

WHICH-FINALLY-HALTS.--AT-THAT-POINT
  AT-THAT

WHICH-FINALLY-HALTS.--AT-THAT-POINT
   AT-THAT

WHICH-FINALLY-HALTS.--AT-THAT-POINT
    AT-THAT

WHICH-FINALLY-HALTS.--AT-THAT-POINT
     AT-THAT

WHICH-FINALLY-HALTS.--AT-THAT-POINT
      AT-THAT

WHICH-FINALLY-HALTS.--AT-THAT-POINT
       AT-THAT

WHICH-FINALLY-HALTS.--AT-THAT-POINT
        AT-THAT

WHICH-FINALLY-HALTS.--AT-THAT-POINT
         AT-THAT

WHICH-FINALLY-HALTS.--AT-THAT-POINT
          AT-THAT

WHICH-FINALLY-HALTS.--AT-THAT-POINT
           AT-THAT

WHICH-FINALLY-HALTS.--AT-THAT-POINT
              AT-THAT

WHICH-FINALLY-HALTS.--AT-THAT-POINT
               AT-THAT

WHICH-FINALLY-HALTS.--AT-THAT-POINT
                AT-THAT

WHICH-FINALLY-HALTS.--AT-THAT-POINT
                 AT-THAT

WHICH-FINALLY-HALTS.--AT-THAT-POINT
                      AT-THAT



(22, True)

### A retenir

1. L'Algorithme de boyer-moore est un algorithme de recherche textuelle
2. Basé sur un pré traitement du motif avec un calcul de tables de décalage
3. Il compare le texte et le motif en partant par la fin du motif. Si les lettres ne correspondent pas; il décale le motif
4. Amélioration de type 5 fois mieux qu'un algo de recherche de base
5. Pas terrible si peu de lettre dans l'alphabet
